In [ ]:
#Simple CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

class SimpleCNN:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize model with input shape
        model = Sequential()
        input_shape = (height, width, depth)
        chan_dim = -1
        
        # first CONV=>RELU=>BN layer set
        model.add(Conv2D(32, (3,3), strides=(2,2), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # Second CONV=>RELU=>BN layer set
        model.add(Conv2D(64, (3,3), strides=(2,2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        
        # First and only set of FC=>RELU layers
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))
        
        return model

In [ ]:
# Generate Adversarial image using FGSM
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary(model, image, label, eps=2/255.0):
    # cast the image
    image = tf.cast(image, tf.float32)
    
    # record gradients
    with tf.GradientTape() as tape:
        # explicitly indicate that our image should be tracked for growing gradients
        tape.watch(image)
        
        # use model to make predictions on input image and compute loss
        pred = model(image)
        loss = MSE(label, pred)
        
    # calculate gradients of loss w.r.t image then compute sign of gradient
    gradient = tape.gradient(loss, image)
    signed_grad = tf.sign(gradient)
    
    # construct the image adversary
    adversary = (image + (signed_grad * eps)).numpy()
    
    return adversary

In [ ]:
# Generating images batches (normal+adversarial)
from sklearn.utils import shuffle
import numpy as np

# generating adversary images batch
def generate_adversarial_batch(model, total, images, labels, dims, eps=0.01):
    # unpack image dimensions into convienence variables
    (h, w, c) = dims
    
    # since we are constructing a data generator here so we need to loop indefinitely
    while True:
        
        # initialize our preturbed images and labels
        preturbed_images = []
        preturbed_labels = []
        
        # randomly sample indexes (without replacement) from input data
        idxs = np.random.choice(range(0, len(images)), size=total, replace=False)
            
        # loop over the indexes
        for i in idxs:
                
            # grab current image and label
            image = images[i]
            label = labels[i]
                
            # generate an adversarial image
            adversary = generate_image_adversary(model, image.reshape(1, h, w, c),
                                                    label, eps)
                
            # update preturbed images and label lists
            preturbed_images.append(adversary.reshape(h, w, c))
            preturbed_labels.append(label)
                
        #yield preturbed images and labels
        yield (np.array(preturbed_images), np.array(preturbed_labels))

# generating mixed images batch
def generate_mixed_adversarial_batch(model, total, images, labels, dims, eps=0.01, split=0.5):
  # unpack image dimensions into convienence variables
  (h, w, c) = dims

  #compute the total number of training images to keep along with the number of adversarial images 
  # to generate
  total_normal = int(total*split)
  total_adv = int(total * (1 - split))

  # since we are constructing a data generator so we need to loop indefinatly
  while True:

    # randomly sample the indexes (without replacement) from the data and thenuse those indexes to 
    # sample our normal images and labels
    idxs = np.random.choice(range(0, len(images)), size=total_normal, replace=False)
    mixed_images = images[idxs]
    mixed_labels = labels[idxs]

    # again randomly sample indexes from the input data, this time to construct our adversarial 
    # images
    idxs = np.random.choice(range(0, len(images)), size=total_adv, replace=False)
    
    # loop over the indexes
    for i in idxs:
      
      # grab current image and label, then use that data to generate adversarial example
      image = images[i]
      label = labels[i]
      adversary = generate_image_adversary(model, image.reshape(1, h, w, c), label, eps=eps)

      # update mixed images and labels list
      mixed_images = np.vstack([mixed_images, adversary])
      mixed_labels = np.vstack([mixed_labels, label])

    # shuffle images and labels
    (mixed_images, mixed_labels) = shuffle(mixed_images, mixed_labels)

    # yield the mixed images and labels to calling function
    yield (mixed_images, mixed_labels)

In [ ]:
# training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2 as cv

# load mnist dataset and scale the pixel values to range [0, 1]
print('[INFO] loading MNIST dataset...')
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0

# add a chennel dimension to images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

# one hot encode labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize optimizer and model
print('[INFO] Compiling model...')
opt = Adam(lr=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# train simple CNN on MNIST
print('[INFO] Training Model...')
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, 
          epochs=20, verbose=1)

# make predictions on testing set for model trained on non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] loading MNIST dataset...
11501568/11490434 [==============================] - 0s 0us/step
[INFO] Compiling model...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[INFO] Training Model...
Epoch 1/20
938/938 [==============================] - 21s 6ms/step - loss: 0.1937 - accuracy: 0.9413 - val_loss: 0.0615 - val_accuracy: 0.9795
Epoch 2/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0806 - accuracy: 0.9752 - val_loss: 0.0460 - val_accuracy: 0.9842
Epoch 3/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0578 - accuracy: 0.9822 - val_loss: 0.0390 - val_accuracy: 0.9871
Epoch 4/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0483 - accuracy: 0.9849 - val_loss: 0.0394 - val_accuracy: 0.9858
Epoch 5/20
938/938 [==============================] - 5s 6ms/step - loss: 0.0383 - accuracy: 0.9877 - val_loss: 0.0408 - val_accuracy: 0.9869
Epoch 6/20
938/938 [==============================] - 5s 5ms/step - loss: 0.0329 - accuracy: 0.9896 - val_loss: 0.0382 - val_accuracy: 0.9880
Epoch 7/20
938/938 [==============================] - 5s 5ms/step - loss: 0.0313 - accuracy: 0.9900 - val_loss: 0.0377 - v

In [ ]:
# Testing on adversarial images

# generate a set of adversarial from our test set
print('[INFO] generating adversarial from our test set')
(advX, advY) = next(generate_adversarial_batch(model, len(testX), testX, testY, 
                                               (28, 28, 1), eps=0.1))

# re-evaluate model on adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print('[INFO] adversarial testing images:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

# since accuracy has dropped on the adversarial images we will fine-tune our model 
#on the adversarial images
# lower learning rate and re-compile model such that we can fine tune it on the mixed batches of 
# normal images and dynamically generated adverserial images
print('[INFO] re-compiling model...')
opt = Adam(lr = 1e-4)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# initialize our data generator to create data batches containing a mix of both normal images and
# adversarial images
print('[INFO] creating mixed data generator')
data_gen = generate_mixed_adversarial_batch(model, 64, trainX, trainY, (28, 28, 1), 
                                            eps=0.1, split=0.5)

# train model on mixed images
print('[INFO] Training network on dynamic mixed images...')
model.fit(data_gen, steps_per_epoch=len(trainX)//64, epochs=15, verbose=1)


# now that our model is re-trained we should evaluate it on test set (non-adversarial)
# to check if performance has degraded
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print('')
print('[INFO] testing on normal images after re-training:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

# final evaluation of model on adversarial images
(loss, acc) = model.evaluate(x=advX, y=advY, verbose=0)
print('[INFO] testing on adversarial images after re-training:')
print(f'[INFO] loss: {loss:.4f}, accuracy: {acc:.4f}')

[INFO] generating adversarial from our test set
[INFO] adversarial testing images:
[INFO] loss: 14.3745, accuracy: 0.0141
[INFO] re-compiling model...
[INFO] creating mixed data generator
[INFO] Training network on dynamic mixed images...


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/15
937/937 [==============================] - 292s 312ms/step - loss: 1.6165 - accuracy: 0.7545
Epoch 2/15
937/937 [==============================] - 293s 313ms/step - loss: 0.4162 - accuracy: 0.8808
Epoch 3/15
937/937 [==============================] - 293s 312ms/step - loss: 0.2842 - accuracy: 0.9110
Epoch 4/15
937/937 [==============================] - 291s 311ms/step - loss: 0.2247 - accuracy: 0.9300
Epoch 5/15
937/937 [==============================] - 291s 310ms/step - loss: 0.1975 - accuracy: 0.9369
Epoch 6/15
937/937 [==============================] - 291s 311ms/step - loss: 0.1713 - accuracy: 0.9453
Epoch 7/15
937/937 [==============================] - 291s 310ms/step - loss: 0.1589 - accuracy: 0.9486
Epoch 8/15
937/937 [==============================] - 291s 311ms/step - loss: 0.1451 - accuracy: 0.9529
Epoch 9/15
937/937 [==============================] - 292s 312ms/step - loss: 0.1319 - accuracy: 0.9566
Epoch 10/15
937/937 [==============================] - 292s 312m